In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker as tk
import scipy as sp
from struct import unpack as unp


conditions = ["popAvg", "ogTrace", "slowTrace", "midTrace", "fastTrace"]

In [ ]:
# It is important that these be arrays because they need to be
# in the same order as the parameters.h file
pops = ["V1L4E", "V1L4I", "V1L23E", "V1L23I", "V2L4E", "V2L4I", "V2L23E", "V2L23I", "LGN"]
dims = {"V1L4E":[23,4], "V1L4I":[23,1], "V1L23E":[17,7], "V1L23I":[17,2], "V2L4E":[13,10], "V2L4I":[13,3], "V2L23E":[10,16], "V2L23I":[10,5], "LGN":[32,2]}
RFsv = {"FF":{"V1L4":10, "V1L23":7, "V2L4":5, "V2L23":4}, "Lat":{"V1L4":11, "V1L23":9, "V2L4":7, "V2L23":5}, "FB":{"V1L4":7, "V1L23":8, "V2L4":4, "V2L23":0}}
syns = [ ["LGN", "V1L4E", "FF"], ["LGN", "V1L4I", "FF"], ["V1L4E", "V1L23E", "FF"], ["V1L4E", "V1L23I", "FF"], 
    ["V1L23E", "V2L4E", "FF"], ["V1L23E", "V2L4I", "FF"], ["V2L4E", "V2L23E", "FF"], ["V2L4E", "V2L23I", "FF"],
    ["V1L4E", "V1L4I", "Lat"], ["V1L4I", "V1L4E", "Lat"], ["V1L4I", "V1L4I", "Lat"], ["V1L23E", "V1L23I", "Lat"],
    ["V1L23I", "V1L23E", "Lat"], ["V1L23I", "V1L23I", "Lat"], ["V2L4E", "V2L4I", "Lat"], ["V2L4I", "V2L4E", "Lat"], 
    ["V2L4I", "V2L4I", "Lat"], ["V2L23E", "V2L23I", "Lat"], ["V2L23I", "V2L23E", "Lat"], ["V2L23I", "V2L23I", "Lat"], 
    ["V2L23E", "V1L23E", "FB"], ["V2L23E", "V1L23I", "FB"],  ["V1L23E", "V1L4I", "FB"], ["V2L23E", "V2L4I", "FB"], 
    ["V1L4I", "V1L23E", "FF"], ["V1L4I", "V1L23I", "FF"], ["V2L4I", "V2L23E", "FF"], ["V2L4I", "V2L23I", "FF"] ]
sNames = [ s[0]+"-"+s[1] for s in syns]

def popDim(pop):
    return dims[pop][0]*dims[pop][0]*dims[pop][1]

#  Training Results

In [ ]:
def loadPopRecording(cond, it, pop, noRec = 201):
    with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+pop, mode="rb") as f:
        dim = popDim(pop)
        data = unp("f"*dim*noRec, f.read(4*dim*noRec))
        return np.reshape(np.array(data), (noRec,dim))

def loadWeightRecording(cond, it, src, trg, noRec = 201):
    with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+src+"-"+trg, mode="rb") as f:
        dim = popDim(src) * popDim(trg)
        data = unp("f"*dim*noRec, f.read(4*dim*noRec))
        return np.reshape(np.array(data), (noRec, dim))

## Activity

In [ ]:
# Smoothed Average Activities
for cond in conditions:
    avgR = {p:np.zeros(200) for p in pops}
    for it in range(5):
        for p in pops:
            rates = loadPopRecording(cond, it, p)
            avgR[p] += rates.sum(axis=1)[1:]
            plt.plot(sp.signal.savgol_filter(rates.mean(axis=1)[1:], 20, 2), label = p)
        
        plt.savefig("./RESULTS/"+cond+"/plots/Smoothed Average Rate - "+cond+" - It"+str(it), bbox_extra_artists=(plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)),), bbox_inches='tight')
        plt.show()
    for p in pops:
        plt.plot(sp.signal.savgol_filter(avgR[p]/(popDim(p)*5), 20, 2), label = p)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.savefig("./RESULTS/"+cond+"/plots/Smoothed Average Rate - "+cond, bbox_extra_artists=(plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)),), bbox_inches='tight')
    plt.show()

In [ ]:
# Smoothed Signal Retention
for cond in conditions:
    avgR = {p:np.zeros(200) for p in pops}
    for it in range(5):
        LGNrates = loadPopRecording(cond, it, "LGN")
        avgR["LGN"] += LGNrates.sum(axis=1)[1:]
        LGNrates = LGNrates.mean(axis=1)[1:]
        for p in pops[:-1]:
            rates = loadPopRecording(cond, it, p)
            avgR[p] += rates.sum(axis=1)[1:]
            plt.plot(sp.signal.savgol_filter(rates.mean(axis=1)[1:]/LGNrates, 20, 2), label = p)
        plt.savefig("./RESULTS/"+cond+"/plots/Smoothed Signal Retention - "+cond+" - It"+str(it), bbox_extra_artists=(plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)),), bbox_inches='tight')
        plt.show()
    avgR["LGN"] = avgR["LGN"] / (popDim("LGN")*5)
    for p in pops[:-1]:
        avgR[p] = avgR[p] / (popDim(p)*5)
        plt.plot(sp.signal.savgol_filter(avgR[p]/avgR["LGN"], 20, 2), label = p)
    plt.savefig("./RESULTS/"+cond+"/plots/Smoothed Signal Retention - "+cond, bbox_extra_artists=(plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)),), bbox_inches='tight')
    plt.show()

In [ ]:
# Average tail 5%
data = {"Condition":[], "Iteration":[], "Population":[], "Mean":[], "std":[], }
for cond in conditions:
    aggregate = {p:[] for p in pops[:-1]}
    for it in range(5):
        for p in pops[:-1]:
            rates = loadPopRecording(cond, it, p).mean(axis=1)[-10:]
            aggregate[p].extend(rates)
            data["Mean"].append(rates.mean())
            data["std"].append(rates.std())
            data["Condition"].append(cond)
            data["Iteration"].append(it)
            data["Population"].append(p)
    for p in pops[:-1]:
        data["Mean"].append(np.array(aggregate[p]).mean())
        data["std"].append(np.array(aggregate[p]).std())
        data["Condition"].append(cond)
        data["Iteration"].append(-1)
        data["Population"].append(p)
pd.DataFrame(data).to_csv("./RESULTS/TrainingTailAverage.csv")

In [ ]:
# Activity Distributions
for cond in conditions:
    fig, axs = plt.subplots(len(pops[:-1]),4, figsize=(20,10))
    toshow = [1, 50, 100, 200]
    for k,p in enumerate(pops[:-1]):        
        axs[k][0].text(-.2,.5,p, transform = axs[k][0].transAxes)
        rates = np.concatenate([loadPopRecording(cond, it, p) for it in range(5)], axis = 1)
        for i, t in enumerate(toshow):
            c, b, _ = axs[k][i].hist(rates[t][rates[t].nonzero()], bins = 100, align="mid")
#            axs[k][i].bar(h["bins"][t][1:-1], h["count"][t][1:], width=h["bins"][t][1]*.8, align='center')
            axs[k][i].bar(rates[t][rates[t].nonzero()].mean(), max(c), width = (b[2]-b[1])*.6, color = "red")
#            axs[k][i].bar(rAvg[v][p].values[t], max(h["count"][t][1:]), width=h["bins"][t][1]*.6, color="red")
            perc = 100*(len(rates[t].nonzero()[0])/rates[t].shape[0])
            axs[k][i].text(.9,.8, str(round(perc,2))+"%" if perc >= .01  else "< .01%", fontsize=8,transform=axs[k][i].transAxes)

    for i, t in enumerate(toshow): axs[0][i].set_title(str(t*1000) + " patches")
    fig.tight_layout()
    fig.savefig("./RESULTS/"+cond+"/plots/Activity Distributions")
    fig.show()

In [ ]:
# EI Balance
for cond in conditions:
    fig, axs = plt.subplots(2,2, figsize = (10, 5))
    for i,p in enumerate(["V1L4", "V1L23", "V2L4", "V2L23"]):
        for it in range(5):
            exc = loadPopRecording(cond, it, p+"E").sum(axis=1)
            inh = loadPopRecording(cond, it, p+"I").sum(axis=1)
#            inh[inh == 0] = np.nextafter(0,1)
            inh[inh == 0] = np.min(inh[inh.nonzero()])

            bal = exc / inh
            bal[bal > 50] = 50 #Manual cutoff
#            axs.flatten()[i].plot(sp.signal.savgol_filter(bal, 20, 2))
            axs.flatten()[i].plot(bal)
        axs.flatten()[i].set_title(p)
    fig.tight_layout()
    fig.savefig("./RESULTS/"+cond+"/plots/EI Balance")
    fig.show()

In [ ]:
# EI Balance
# TODO: try the opposite as well to check cause it becomes too small to see
for cond in conditions:
    fig, axs = plt.subplots(2,2, figsize = (10, 5))
    for i,p in enumerate(["V1L4", "V1L23", "V2L4", "V2L23"]):
        bal = []
        for it in range(5):
            exc = loadPopRecording(cond, it, p+"E").sum(axis=1)
            inh = loadPopRecording(cond, it, p+"I").sum(axis=1)
#            inh[inh == 0] = np.nextafter(0,1)
            inh[inh == 0] = np.min(inh[inh.nonzero()])
            b = exc / inh
            b[b > 50] = 50 #Manual cutoff
            bal.append(b)        
        axs.flatten()[i].plot(np.array(bal).mean(axis=0))
        axs.flatten()[i].set_title(p)
    fig.tight_layout()
    fig.savefig("./RESULTS/"+cond+"/plots/Average EI Balance")
    fig.show()

## Plastic Parameters

In [ ]:
# TODO: this is useless, but also wrong cause weights should be architecture-masked first
# Average weights
for cond in conditions:
    for i,src in enumerate(pops):
        for s in syns:
            if s[0] != src: continue
            dim = popDim(src) * popDim(s[1])
            color = next(plt.gca()._get_lines.prop_cycler)['color']
            for it in range(5):
                weights = []
                with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+src+"-"+s[1], mode="rb") as f:
                    for i in range(201):
                        weights.append(np.array(unp("f"*dim, f.read(4*dim))).mean())
                plt.plot(weights, label = s[1] if it == 0 else None, color = color)
        plt.title(src)
        plt.legend(title = "Target")
        plt.savefig("./RESULTS/"+cond+"/plots/Average weights from "+src)
        plt.show()

In [ ]:
# Weights Distributions
toshow = [1, 50, 100, 200]
for s in syns:
    dim = popDim(s[0]) * popDim(s[1])
    mat = np.loadtxt("./rateModel/architecture/matrices/"+s[0]+"-"+s[1]).flatten()
    weights = {t:[] for t in toshow}
    for it in range(5):
        with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+s[0]+"-"+s[1], mode="rb") as f:
            for i in range(201):
                cont = f.read(4*dim)
                if i not in toshow: continue
                weights[i].append(np.array(unp("f"*dim, cont))[mat.nonzero()])
    fig, axs = plt.subplots(2,2, figsize=(10, 5))
    form = tk.ScalarFormatter(useMathText=True)
    form.set_scientific(True)
    form.set_powerlimits((-1,1))
    for i, t in enumerate(toshow):
        ws = np.array(weights[t]).flatten()
        c, b, _ = axs.flatten()[i].hist(ws[ws.nonzero()], bins=100, align='mid')
        axs.flatten()[i].bar(ws[ws.nonzero()].mean(), max(c), width = (b[2]-b[1])*.6, color = "red")
#        perc = 100*(len(ws[t].nonzero()[0])/ws[t].shape[0])
#        axs[k][i].text(.9,.8, str(round(perc,2))+"%" if perc >= .01  else "< .01%", fontsize=8,transform=axs[k][i].transAxes)
        
        axs.flatten()[i].set_title(str(t*1000) + " patches")
        axs.flatten()[i].yaxis.set_major_formatter(form)
    fig.suptitle(s[0]+"-"+s[1])
    fig.tight_layout()
    fig.savefig("./RESULTS/"+cond+"/plots/Weights distribution "+s[0]+"-"+s[1])
    fig.show()


'''
# Weights distributions
for v in vars[:1]:
    fig,axs = plt.subplots(int(len(syns)/4), 4, figsize = (15, 20))
    for i,s in enumerate(syns):
        sName = s[0]+"-"+s[1]
        ax = axs.flatten()[i]
        mat = np.loadtxt(rPath+v+"/training/"+sName+"weights.vec")
        mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
        ws = mat[mask !=0]
        ws = ws[ws!=0]
        wM = ws.mean()
        wStd = ws.std()
        ws = ws[ws < wM+wStd*4]
        c, b, _ = ax.hist(ws, 50)
        ax.bar(x=wM,height=c.max(), width = b[1]*.6, color="red")
'''

In [ ]:
# Intrinsic plasticity
form = tk.ScalarFormatter(useMathText=True)
form.set_scientific(True)
form.set_powerlimits((-1,1))
for cond in conditions:
    aAgg = {p:[] for p in pops[:-1]}
    tAgg = {p:[] for p in pops[:-1]}
    for it in range(5):
        with open("./RESULTS/"+cond+"/It"+str(it), mode="rb") as f:
            for p in pops[:-1]:
                dim = popDim(p)
                a = np.array(unp("f"*dim, f.read(4*dim)))
                aAgg[p].append(a)
                t = np.array(unp("f"*dim, f.read(4*dim)))
                tAgg[p].append(t)

                fig, axs = plt.subplots(1, 2, figsize = (10,4))
                c, b, _ = axs[0].hist(a, bins=100, align='mid')
                axs[0].bar(a.mean(), max(c), width = (b[2]-b[1])*.4, color = "red")
                axs[0].yaxis.set_major_formatter(form)

                c,b, _ = axs[1].hist(t, bins=100, align='mid')
                axs[1].bar(t.mean(), max(c), width = (b[2]-b[1])*.4, color = "red")
                axs[1].yaxis.set_major_formatter(form)

                fig.tight_layout()
                fig.savefig("./RESULTS/"+cond+"/plots/IP parameters for It"+str(it)+" - "+p)
                fig.show()
    for p in pops[:-1]:
        a = np.array(aAgg[p]).flatten()
        t = np.array(tAgg[p]).flatten()
        fig, axs = plt.subplots(1, 2, figsize = (10,4))
        c, b, _ = axs[0].hist(a, bins=100, align='mid')
        axs[0].bar(a.mean(), max(c), width = (b[2]-b[1])*.4, color = "red")
        axs[0].yaxis.set_major_formatter(form)
        c, b, _ = axs[1].hist(t, bins=100, align='mid')
        axs[1].bar(t.mean(), max(c), width = (b[2]-b[1])*.4, color = "red")
        axs[1].yaxis.set_major_formatter(form)
        fig.tight_layout()
        fig.savefig("./RESULTS/"+cond+"/plots/IP parameters for "+p)
        fig.show()


In [ ]:
# Intrinsic plasticity (scatter )
for cond in conditions:
    aAgg = {p:[] for p in pops[:-1]}
    tAgg = {p:[] for p in pops[:-1]}
    for it in range(5):
        with open("./RESULTS/"+cond+"/It"+str(it), mode="rb") as f:
            for p in pops[:-1]:
                dim = popDim(p)
                a = np.array(unp("f"*dim, f.read(4*dim)))
                aAgg[p].append(a)
                t = np.array(unp("f"*dim, f.read(4*dim)))
                tAgg[p].append(t)

                fig, axs = plt.subplots(1, 2, figsize = (10,3))
                axs[0].plot(list(range(dim)), a, '.', markersize = 2)
                axs[1].plot(list(range(dim)), t, '.', markersize = 2)
                
                fig.savefig("./RESULTS/"+cond+"/plots/IP scatter for It"+str(it)+" - "+p)
                fig.tight_layout()
                fig.show()
    for p in pops[:-1]:
        dim = popDim(p)
        fig, axs = plt.subplots(1, 2, figsize = (10,3))
        color = next(plt.gca()._get_lines.prop_cycler)['color']
        for a in aAgg[p]:
            axs[0].plot(list(range(dim)), a, '.', markersize = 2, color=color)
        for t in tAgg[p]:
            axs[1].plot(list(range(dim)), t, '.', markersize = 2, color=color)
        fig.tight_layout()
        fig.savefig("./RESULTS/"+cond+"/plots/IP scatter for "+p)
        fig.show()


## Receptive Fields

In [ ]:
# Receptive Fields - complete
for cond in conditions:
    for it in range(5):
        for s in syns[:2]:
            sName = s[0]+"-"+s[1]
            srcDim = popDim(s[0])
            trgDim = popDim(s[1])
            with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+s[0]+"-"+s[1], mode="rb") as f:
                mat = []
                for _ in range(201):
                    mat = f.read(4*srcDim*trgDim)
            mat = np.array(unp("f"*srcDim*trgDim, mat)).reshape((srcDim, trgDim))
            mask = np.loadtxt("./rateModel/architecture/matrices/"+s[0]+"-"+s[1])
            fE, axE = plt.subplots(10, 10, figsize = (10,10))
            fE.suptitle(sName+" On", y=.92)
            fI, axI = plt.subplots(10, 10, figsize = (10,10))
            fI.suptitle(sName+" Off", y=.92)
            fRF, axRF = plt.subplots(10, 10, figsize = (10,10))
            fRF.suptitle(sName+" Receptive Fields", y=.92)
            for i in range(100):
                rf = mat[:,100+i][mask[:,100+i]!=0.].reshape(10,10,2)
                exc = rf[:,:,0]/rf[:,:,0].max()
                inh = rf[:,:,1]/rf[:,:,1].max()
                axE[i//10][i%10].matshow(exc, cmap=plt.cm.Blues)
                axE[i//10][i%10].set_xticks([],[])
                axE[i//10][i%10].set_yticks([],[])
                axI[i//10][i%10].matshow(inh, cmap=plt.cm.Reds)
                axI[i//10][i%10].set_xticks([],[])
                axI[i//10][i%10].set_yticks([],[])
                axRF[i//10][i%10].matshow(inh-exc, cmap=plt.cm.bwr)
                axRF[i//10][i%10].set_xticks([],[])
                axRF[i//10][i%10].set_yticks([],[])
            fE.savefig("./RESULTS/"+cond+"/plots/RFOn "+cond+" - "+"It"+str(it)+" - "+s[1])
            fI.savefig("./RESULTS/"+cond+"/plots/RFOff "+cond+" - "+"It"+str(it)+" - "+s[1])
            fRF.savefig("./RESULTS/"+cond+"/plots/RF "+cond+" - "+"It"+str(it)+" - "+s[1])

In [ ]:
# Receptive Fields - Smaller
for cond in conditions:
    for it in range(5):
        for s in syns[:2]:
            sName = s[0]+"-"+s[1]
            srcDim = popDim(s[0])
            trgDim = popDim(s[1])
            with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+s[0]+"-"+s[1], mode="rb") as f:
                mat = []
                for _ in range(201):
                    mat = f.read(4*srcDim*trgDim)
            mat = np.array(unp("f"*srcDim*trgDim, mat)).reshape((srcDim, trgDim))        
            mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
            fRF, axRF = plt.subplots(10, 10, figsize = (7,7))
            fRF.suptitle(s[1], y=.92)
            for i in range(100):
                rf = mat[:,100+i][mask[:,100+i]!=0.].reshape(10,10,2)
                exc = rf[:,:,0]/rf[:,:,0].max()
                inh = rf[:,:,1]/rf[:,:,1].max()
                if True: # Separate versus joint scaling
                    vrf = inh-exc
                else:
                    vrf = rf[:,:,1]-rf[:,:,0]
                    vrf /= np.abs(vrf.flatten()).max()

                axRF[i//10][i%10].matshow(vrf, cmap=plt.cm.bwr)
                axRF[i//10][i%10].set_xticks([],[])
                axRF[i//10][i%10].set_yticks([],[])
            fRF.savefig("./RESULTS/"+cond+"/plots/RF2 "+cond+" - "+"It"+str(it)+" - "+s[1])

In [ ]:

# Not calculated for slowTrace, midTrace, and fastTrace

from scipy.optimize import curve_fit as cf
import math as m
import random as r
from sklearn.metrics import mean_absolute_error as mae
import warnings
warnings.filterwarnings('ignore')

def gab(c, sx, sy, f, th, phi, A, B):
    x = c%10
    y = np.floor(c/10)
    x1 = x*np.cos(th)+y*np.sin(th)
    y1 = -x*np.sin(th)+y*np.cos(th)
    xp = -(x1**2/(2*sx**2))-(y1**2/(2*sy**2))
    return A*np.exp(xp)*np.cos(2*m.pi*f*x1-phi)+B

def find_ext(x, y):
    try:
        p,_,inf,_,_ = cf(gab,x,y, p0=[r.uniform(0,10),r.uniform(0,10), r.uniform(0,100), 6.28*r.random(), 6.28*r.random(), r.uniform(0,5), r.random()], full_output=True)
        err = mae(y, inf["fvec"])
        return p[0], p[1], p[3], err
    except RuntimeError:
        return 0,0,0,-1

for cond in conditions:
    for s in syns[:2]:
        sName = s[0]+"-"+s[1]
        print("Calculating extents for: "+cond+" - "+sName)
        srcDim = popDim(s[0])
        trgDim = popDim(s[1])
        pars = {"sx":[], "sy":[], "th":[], "err":[], "Iter":[]}
        for it in range(5):
            with open("./RESULTS/"+cond+"/training/It"+str(it)+"_"+s[0]+"-"+s[1], mode="rb") as f:
                mat = []
                for _ in range(201):
                    mat = f.read(4*srcDim*trgDim)
            mat = np.array(unp("f"*srcDim*trgDim, mat)).reshape((srcDim, trgDim))        
            mask = np.loadtxt("rateModel/architecture/matrices/"+sName)
            noRf = mat.shape[1]
            for i in range(noRf):
                if i%100==0: print("  "+str(i)+"/"+str(noRf))
                rf = mat[:,i][mask[:,i]!=0.].reshape(10,10,2)
                exc = rf[:,:,0]/rf[:,:,0].max()
                inh = rf[:,:,1]/rf[:,:,1].max()
                rf = inh-exc
                x = [i for i in range(100)]
                y = rf.flatten()
                sx, sy, th = 0,0,0
                err = 1000000000
                for _ in range(100):
                    nErr = -1
                    noTries = 0
                    while nErr == -1 or noTries==10:
                        Tsx, Tsy, Tth, nErr = find_ext(x, y)
                        noTries+=1
                    if nErr < err and abs(Tsx) <=10 and abs(Tsy)<=10:
                        err = nErr
                        sx = Tsx
                        sy = Tsy
                        th = Tth
                pars["sx"].append(sx)
                pars["sy"].append(sy)
                pars["th"].append(th)
                pars["err"].append(err)
                pars["Iter"].append(it)
        pd.DataFrame(pars).to_csv("./RESULTS/"+cond+"/training/"+sName+"extents.csv", index=False)

In [ ]:
# Still to run
for cond in conditions:
    for it in range(5):
        for s in syns[:2][:-1]:
            sName = s[0]+"-"+s[1]
            tar = s[1]
            p = pd.read_csv("./RESULTS/"+cond+"/training/"+sName+"extents.csv")
            # Sign has no meaning
            p["sx"] = np.absolute(p["sx"].values)
            p["sy"] = np.absolute(p["sy"].values)
            # Extents 
            p.loc[p["sx"] > 10, "sx"] = np.nan
            p.loc[p["sy"] > 10, "sy"] = np.nan
    p[["sx","sy"]]

# Post-Training

In [ ]:
# Elicited activity
ds = "MNIST"
fig, axs = plt.subplots(4, 2, figsize=(8,5))
for k,v in enumerate(vars):
    evPath = "./RESULTS/variations/"+v+"/eval/on/"
    for i,p in enumerate(pops[:4]):
        a = np.loadtxt(evPath+ds+"_test100"+p).flatten()
        fLen = a.shape[0]
        a = a[a!=0]
        c,b,_ = axs[i][k].hist(a, 100)
        axs[i][k].bar(a.mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
        axs[i][k].text(.8, .8, str(round(100*a.shape[0]/fLen,2))+"%", transform=axs[i][k].transAxes)
        axs[i][k].text(.8, .6, "CV: "+str(round(a.std()/a.mean(),2)), transform=axs[i][k].transAxes)
    axs[0][k].set_title(v)
for i,p in enumerate(pops[:4]): axs[i][0].text(-.3, .5, p, transform=axs[i][0].transAxes)
#fig.suptitle(ds)
fig.tight_layout()
'''
ds = "CIFAR-10"
fig, axs = plt.subplots(4, 2, figsize=(8,5))
for k,v in enumerate(vars):
    evPath = "./RESULTS/variations/"+v+"/eval/on/"
    for i,p in enumerate(pops[:4]):
        a = np.loadtxt(evPath+ds+"_test_gray100"+p).flatten()
        fLen = a.shape[0]
        a = a[a!=0]
        c,b,_ = axs[i][k].hist(a, 100)
        axs[i][k].bar(a.mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
        axs[i][k].text(.8, .8, str(round(100*a.shape[0]/fLen,2))+"%", transform=axs[i][k].transAxes)
    axs[0][k].set_title(v)
for i,p in enumerate(pops[:4]): axs[i][0].text(-.3, .5, p, transform=axs[i][0].transAxes)
fig.suptitle(ds)
fig.tight_layout()

ds = "SVHN"
fig, axs = plt.subplots(4, 2, figsize=(8,5))
for k,v in enumerate(vars):
    evPath = "./RESULTS/variations/"+v+"/eval/on/"
    for i,p in enumerate(pops[:4]):
        a = np.loadtxt(evPath+ds+"_test100"+p).flatten()
        fLen = a.shape[0]
        a = a[a!=0]
        c,b,_ = axs[i][k].hist(a, 100)
        axs[i][k].bar(a.mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
        axs[i][k].text(.8, .8, str(round(100*a.shape[0]/fLen,2))+"%", transform=axs[i][k].transAxes)
    axs[0][k].set_title(v)
for i,p in enumerate(pops[:4]): axs[i][0].text(-.3, .5, p, transform=axs[i][0].transAxes)
fig.suptitle(ds)
fig.tight_layout()

ds = "EMNIST"
fig, axs = plt.subplots(4, 2, figsize=(8,5))
for k,v in enumerate(vars):
    evPath = "./RESULTS/variations/"+v+"/eval/on/"
    for i,p in enumerate(pops[:4]):
        a = np.loadtxt(evPath+ds+"_test100"+p).flatten()
        fLen = a.shape[0]
        a = a[a!=0]
        c,b,_ = axs[i][k].hist(a, 100)
        axs[i][k].bar(a.mean(), c.max(),width=(b[1]-b[0])*.3, color="red")
        axs[i][k].text(.8, .8, str(round(100*a.shape[0]/fLen,2))+"%", transform=axs[i][k].transAxes)
    axs[0][k].set_title(v)
for i,p in enumerate(pops[:4]): axs[i][0].text(-.3, .5, p, transform=axs[i][0].transAxes)
fig.suptitle(ds)
fig.tight_layout()'''